1. Загрузите данные в DataFrame, используя функцию read_csv.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV

df = pd.read_csv('house_price.csv')


2. Сколько строк и столбцов в данных? Есть ли пропуски? Для ответа на
вопросы используйте метод info().

In [ ]:
df.info()
print(f"\nстрок: {len(df)}")
print(f"столбцов: {len(df.columns)}")
print("т.к. не все столбцы имеют 1460 non-null значений, значит, есть пропуски")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

3. Для выполнения задания в наборе данных необходимо оставить только
числовые признаки.

In [ ]:
df = df.select_dtypes([np.number])
print(f"осталось столбцов: {len(df.columns)}")

осталось столбцов: 38


4. Удалите столбец Id и пропущенные значения

In [ ]:
df.drop('Id', axis=1, inplace=True)

print("столбцы с пропущенными значениями")
def na_data_calc(df):
  na_count = df.isnull().sum().sort_values(ascending=False)
  na_rate = na_count / len(df)
  na_data = pd.concat([na_count, na_rate],
                      axis=1,
                      keys=['count','ratio'])
  return na_data[na_data["count"] > 0]
print(na_data_calc(df))

# удалим столбцы, где больше 0.15 пропущено
df.drop(na_data_calc(df)[na_data_calc(df)["ratio"] > 0.15].index, axis=1, inplace=True)
print(f"\nосталось столбцов: {len(df.columns)}")

# удалим строки с пропущенными значениями
print(f"\nбыло строк: {len(df)}")
df = df.dropna()
print(f"осталось строк: {len(df)}")

столбцы с пропущенными значениями
             count     ratio
LotFrontage    259  0.177397
GarageYrBlt     81  0.055479
MasVnrArea       8  0.005479

осталось столбцов: 36

было строк: 1460
осталось строк: 1371


5. Разделите набор данных на входные данные X (все столбцы кроме
SalePrice) и ответы y (столбец SalePrice).

6. Разделите данные на обучающую и тестовую выборки.

In [ ]:
dfy = df["SalePrice"]
dfx = df.loc[:, df.columns != "SalePrice"]
dfx_train, dfx_test, dfy_train, dfy_test = train_test_split(dfx, dfy, test_size=0.3)
print("количество строк для обучения:", len(dfx_train))
print("количество строк для тестирования:", len(dfy_test))

количество строк для обучения: 959
количество строк для тестирования: 412


7. Обучите модель LassoCV (установите значение гиперпараметра cv
самостоятельно). Оцените качество полученной модели. Посмотрите на
коэффициенты модели. Есть ли коэффициенты равные 0? Что это означает?

In [ ]:
regl1 = LassoCV(cv=5).fit(dfx_train, dfy_train)
regl1_sc = regl1.score(dfx_test, dfy_test)
print("LassoCV score:", round(regl1_sc, 3))
print("LassoCV coef:")
print(regl1.coef_)
print("есть нулевые коэффициенты?", not all(regl1.coef_))
print("они означают, что соответствующие столбцы не важны для модели.")

LassoCV score: 0.761
LassoCV coef:
[ -0.           0.33405324   0.           0.         282.53646073
 144.37320273  45.26884842   7.23998718  -0.           0.
  33.24310785   0.           0.          -0.          61.50593129
   0.          -0.           0.           0.          -0.
  -0.           0.           0.           0.           0.
  62.07405213  29.63779682   0.          -0.           0.
  17.10690114 -23.26905842  -0.33380935   0.          -0.        ]
есть нулевые коэффициенты? True
они означают, что соответствующие столбцы не важны для модели.


Попробуйте их убрать и построить модель заново. Как изменилось качество
полученной модели?

In [ ]:
dfx2 = dfx[[col for i, col in enumerate(dfx.columns) if regl1.coef_[i] != 0]]
dfx2_train, dfx2_test, dfy2_train, dfy2_test = train_test_split(dfx2, dfy, test_size=0.3)

reg2l1 = LassoCV(cv=5).fit(dfx2_train, dfy2_train)
reg2l1_sc = reg2l1.score(dfx2_test, dfy2_test)
print("LassoCV score:", round(reg2l1_sc, 3))
print("LassoCV coef:")
print(reg2l1.coef_)

print("\nкачество модели",
      "снизилось" if (reg2l1_sc < regl1_sc) else "улучшилось",
      "на", round(abs(reg2l1_sc - regl1_sc), 3))

LassoCV score: 0.758
LassoCV coef:
[ 3.12040630e-01  3.65191441e+02  9.29075320e+01  1.78520786e+01
  4.37910494e+00  3.13100161e+01  6.46379707e+01  7.17047006e+01
  2.99560513e+01  3.77271178e+01 -0.00000000e+00 -8.46823196e-02]

качество модели снизилось на 0.002


8. Попробуйте использовать L2-регуляризацию, т.е. обучите модель RidgeCV.
Сравните полученный результат LassoCV и RidgeCV.

In [ ]:
regl2 = RidgeCV().fit(dfx_train, dfy_train)
regl2_sc = regl2.score(dfx_test, dfy_test)
print("RidgeCV score:", round(regl2_sc, 3))
print("по коэффициенту детерминации лучше",
      "RidgeCV" if regl2_sc > reg2l1_sc else "LassoCV")

RidgeCV score: 0.802
по коэффициенту детерминации лучше RidgeCV


Дополнительно: используя MinMaxScaler приведите данные к одному масштабу и попробуйте использовать полином 2-й степени с L2 регуляризацией (Ridge). Получите коэффициент детерминации для тестового и обучающего наборов данных.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures  # Класс преобразователь.

scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

dfsy = df_scaled["SalePrice"]
dfsx = df_scaled.loc[:, df_scaled.columns != "SalePrice"]
dfsx_train, dfsx_test, dfsy_train, dfsy_test = train_test_split(dfsx, dfsy, test_size=0.3)

quad = PolynomialFeatures(degree=2)
dfsxq_train = quad.fit_transform(dfsx_train)
dfsxq_test = quad.fit_transform(dfsx_test)

reg2l2 = RidgeCV().fit(dfsxq_train, dfsy_train)
reg2l2_sc = reg2l2.score(dfsxq_test, dfsy_test)
print("RidgeCV score:", round(reg2l2_sc, 3))
print("по коэффициенту детерминации лучше",
      "RidgeCV 1 степени" if regl2_sc > reg2l2_sc else "RidgeCV 2 степени с нормализацией")

RidgeCV score: 0.872
по коэффициенту детерминации лучше RidgeCV 2 степени с нормализацией
